# IUM Projekt - etap 2

Vladyslav Kyryk, Michał Łątkowski

# Zadanie 8

Mamy problemy odpowiednim zapełnianiem półek magazynowych. Nigdy nie wiadomo, co tak naprawdę będzie potrzebne w najbliższym tygodniu, co powinniśmy zamówić. Może da się coś z tym zrobić?”

## Modele

Aplikacja udostępnia dwa modele:
1. Model bazowy oparty na... TODO
2. Oparty na regresji za pomocą XGBoost model, który jest naszym modelem "zaawansowanym"

### Model 

#### Trening

Model można wytrenować uruchamiając notebook <em>xgboost</em>.

#### Atrybuty

W celu trenowania do modelu podaliśmy dane dotyczące:
<ul>
  <li>Ceny produktu</li>
  <li>Kategorii produktu</li>
  <li>Zmienne oparte na dacie (miesiąc, tydzień, rok etc.)</li>
</ul>

Próbowaliśmy również atrybutów takich jak miesięczna średnia, ale nie zmieniało to w znaczący sposób wyników modelu (opieraliśmy się na mierze RMSE).
Nie chcieliśmy zatem komplikować modelu atrybutami niewnoszącymi zbyt wiele

#### ARIMA

Pierwotnie planowaliśmy zastosować model ARIMA albo, po późniejszej konkluzji, że ARIMA nie radzi sobie z trendami sezonowymi występującymi w danych, SARIMA. Efekty naszej pracy w tym zakresie można zobaczyć szczegółowo w notebooku <em>arima</em>

## Mikroserwis

Aplikacja została oparta o bibliotekę Flask. Użytkownik wykonuje zapytanie z odpowiednimi parametrami na jeden z endpointów.
<em>/predict/basic_model</em> TODO ...
albo
<em>/predict/advanced_model</em> W ciele metody GET podajemy dwa parametry - <em>date</em>, w postaci np. "2021-10-21" oraz <em>product_id</em> w postaci np. "1002".
Aplikacja wczytuje model uprzednio zapisany przez notebook treningowy, pobiera dane bazując na tym co podał użytkownik, przygotowuje te dane (tak żeby model dostał na wejściu dane w formacie, którego się "spodziewa"), a potem 
Aplikacja w obu przypadkach zwraca liczbę całkowitą będącą równą przewidywanej ilości sprzedanych egzemplarzy.

#### Uruchomienie aplikacji

Aplikację można uruchomić, wpisując (jeśli jest się w folderze z aplikacją) python3 application.py.

Przykładowe zapytanie do modelu bazowego: http://127.0.0.1:5000/predict/basiC
Przykładowa odpowiedź modelu bazowego: 

Przykładowe zapytanie do modelu zaawansowanego: http://127.0.0.1:5000/predict/advanced_model?date=2021-10-21&product_id=1002
Przykładowa odpowiedź modelu zaawansowanego: 17

## TODO A/B TESTY